In [0]:
# ============================================================
# CELL 1 – Imports and config
# ============================================================

from pyspark import pipelines as pl
from pyspark.sql.functions import *
from pyspark.sql.types import *

# All 4 JSON files are already here:
volume_path = "/Volumes/workspace/damg7370/datastore/team_assign//customer_*.json"
# ============================================================
# CELL 2 – BRONZE (RESCUE MODE)
# ============================================================

pl.create_streaming_table("demo_cust_bronze_sd")

@pl.append_flow(
    target = "demo_cust_bronze_sd",
    name   = "demo_cust_bronze_sd_ingest_flow"
)
def demo_cust_bronze_sd_ingest_flow():
    df = (
        spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "json")
            .option("cloudFiles.inferColumnTypes", "true")
            .option("cloudFiles.schemaEvolutionMode", "rescue")
            .load(volume_path)
    )

    df = (
        df
        .withColumn("ingestion_datetime", current_timestamp())
        .withColumn("source_filename", col("_metadata.file_path"))
    )

    return df
# ============================================================
# CELL 3 – DATATYPE CHANGE HELPER (streaming-safe)
#          e.g. SignupDate string -> date
# ============================================================

def process__rescue_data_datatype_change(df, updated_schema: StructType):
    """
    For each field in updated_schema:
      - If column does not exist, create it from _rescued_data.
      - If it exists, override with rescued value when present,
        otherwise cast existing column.
      - Finally clear _rescued_data.
    """

    if "_rescued_data" not in df.columns:
        return df

    # Parse rescued JSON into map<string,string>
    df = df.withColumn(
        "_rescued_data_map",
        from_json(col("_rescued_data"), MapType(StringType(), StringType()))
    )

    for field in updated_schema:
        col_name  = field.name
        data_type = field.dataType

        rescued_val = col("_rescued_data_map").getItem(col_name)

        if col_name in df.columns:
            # prefer rescued value when not null
            df = df.withColumn(
                col_name,
                when(rescued_val.isNotNull(), rescued_val.cast(data_type))
                  .otherwise(col(col_name).cast(data_type))
            )
        else:
            # column didn’t exist before → create from rescued data
            df = df.withColumn(
                col_name,
                rescued_val.cast(data_type)
            )

    df = df.drop("_rescued_data_map")

    # we used rescued data, now clear it (for expectations)
    df = df.withColumn("_rescued_data", lit(None).cast(StringType()))

    return df
# ============================================================
# CELL 4 – NEW FIELDS HELPER (streaming-safe, no collect/count)
#          handles Age, Gender, LoyaltyStatus, CreditScore
# ============================================================

def process__rescue_data_new_fields(df):
    """
    Streaming-safe:
    - Parse _rescued_data into a map
    - For known new fields, create / update columns from that map.
    """

    if "_rescued_data" not in df.columns:
        return df

    df = df.withColumn(
        "_rescued_data_json_to_map",
        from_json(col("_rescued_data"), MapType(StringType(), StringType()))
    )

    # New fields we expect in this assignment
    new_fields = [
        ("Age", IntegerType()),
        ("Gender", StringType()),
        ("LoyaltyStatus", StringType()),
        ("CreditScore", IntegerType())
    ]

    for col_name, data_type in new_fields:
        rescued_val = col("_rescued_data_json_to_map").getItem(col_name)

        if col_name in df.columns:
            df = df.withColumn(
                col_name,
                when(rescued_val.isNotNull(), rescued_val.cast(data_type))
                  .otherwise(col(col_name).cast(data_type))
            )
        else:
            df = df.withColumn(
                col_name,
                rescued_val.cast(data_type)
            )

    # keep _rescued_data for the datatype-change step
    return df
# ============================================================
# CELL 5 – TARGET DATATYPE SCHEMA
#          NOTE: correct column name is "SignupDate"
# ============================================================

updated_datatypes = StructType([
    StructField("SignupDate", DateType(), True)
])
# ============================================================
# CELL 6 – SILVER (RESCUE MODE) – main assignment pipeline
#          This fixes the workshop bug (CreditScore + SignupDate)
# ============================================================

pl.create_streaming_table(
    name = "demo_cust_silver_sd",
    expect_all_or_drop = {
        "no_rescued_data": "_rescued_data IS NULL",
        "valid_id": "CustomerID IS NOT NULL"
    }
)

@pl.append_flow(
    target = "demo_cust_silver_sd",
    name   = "demo_cust_silver_sd_clean_flow"
)
def demo_cust_silver_sd_clean_flow():
    df = spark.readStream.table("demo_cust_bronze_sd")

    # 1) New fields from _rescued_data (Age, Gender, LoyaltyStatus, CreditScore)
    df = process__rescue_data_new_fields(df)

    # 2) Datatype change (SignupDate -> DateType) + clear _rescued_data
    df = process__rescue_data_datatype_change(df, updated_datatypes)

    return df
# ============================================================
# CELL 7 – BRONZE (ADDNEWCOLUMNS MODE)
# ============================================================

pl.create_streaming_table("demo_cust_bronze_sd_addcols")

@pl.append_flow(
    target = "demo_cust_bronze_sd_addcols",
    name   = "demo_cust_bronze_sd_addcols_ingest_flow"
)
def demo_cust_bronze_sd_addcols_ingest_flow():
    df = (
        spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "json")
            .option("cloudFiles.inferColumnTypes", "true")
            .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
            .load(volume_path)
    )

    df = (
        df
        .withColumn("ingestion_datetime", current_timestamp())
        .withColumn("source_filename", col("_metadata.file_path"))
    )

    return df
# ============================================================
# CELL 8 – SILVER (ADDNEWCOLUMNS MODE)
#          No rescued-data logic: new columns come automatically
# ============================================================

pl.create_streaming_table(
    name = "demo_cust_silver_sd_addcols",
    expect_all_or_drop = {
        "valid_id": "CustomerID IS NOT NULL"
    }
)

@pl.append_flow(
    target = "demo_cust_silver_sd_addcols",
    name   = "demo_cust_silver_sd_addcols_clean_flow"
)
def demo_cust_silver_sd_addcols_clean_flow():
    df = spark.readStream.table("demo_cust_bronze_sd_addcols")

    # Just normalize SignupDate if it exists
    if "SignupDate" in df.columns:
        df = df.withColumn("SignupDate", to_date(col("SignupDate")))

    return df


Age,City,CreditScore,CustomerID,Email,FullName,Gender,LoyaltyStatus,PhoneNumber,SignupDate,_rescued_data,ingestion_datetime,source_filename,_rescued_data_json_to_map
null,New York,null,C001,alice.j@example.com,Alice Johnson,null,null,555-123-4567,2023-01-15,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_1.json,null
null,Chicago,null,C002,bob.smith@example.com,Bob Smith,null,null,555-234-5678,2023-02-20,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_1.json,null
null,San Diego,null,C003,carol.lee@example.com,Carol Lee,null,null,555-345-6789,2023-03-05,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_1.json,null
null,Austin,null,C004,david.kim@example.com,David Kim,null,null,555-456-7890,2023-04-12,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_1.json,null
null,Dallas,null,C010,jack.n@example.com,Jack Nguyen,null,null,555-012-3456,2023-10-21,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_1.json,null
26,New York,null,C001,alice.johnson@example.com,Alice Johnson,Female,Platinum,555-116-7521,2023-02-28,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_2.json,null
58,Chicago,null,C002,bob.smith@example.com,Bob Smith,Male,Silver,555-534-5537,2023-08-04,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_2.json,null
34,San Diego,null,C003,carol.lee@example.com,Carol Lee,Female,Platinum,555-524-5491,2023-05-24,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_2.json,null
66,Austin,null,C004,david.kim@example.com,David Kim,Non-binary,Bronze,555-557-5139,2023-03-11,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_2.json,null
34,Seattle,null,C005,eva.martinez@example.com,Eva Martinez,Female,Platinum,555-384-8895,2023-04-05,null,2025-11-16T04:20:18.851Z,/Volumes/workspace/damg7370/datastore/schema_drift/demo_smm/customer_data_2.json,null
